In [13]:
import pandas as pd, country_converter as coco

In [27]:
# Open .xlsx file
df = pd.read_excel('vat_substandard_rates.xlsx', skiprows=3)

# Remove bottom 11 rows
df = df[:-11].copy()

# Rename 'Unnamed: 0' to 'Country'
df.rename(columns={'Unnamed: 0': 'Country'}, inplace=True)

# Add 'ISO3' column
df['ISO3'] = coco.convert(names=df['Country'], to='ISO3')

# Add continent column
df['Continent'] = coco.convert(names=df['Country'], to='continent')

# Drop if not found
df = df[df['ISO3'] != 'not found'].copy()

# Filter to Continent
df = df[df['Continent'] == 'Europe'].reset_index(drop=True).copy()

Channel Islands not found in regex
Channel Islands not found in regex


In [40]:
# Clean `VAT/GST (standard)` column. Rules:
# If value is percentage (e.g. 20%), convert to decimal (e.g. 0.20)
# If value us 'no', convert to 0
# If value starts with percentage, then has other text, take the first percentage and convert to decimal
# If value is NaN, convert to 0

def clean_vat(row):
    if row['VAT/GST (standard)'] == 'no':
        return 0
    elif pd.isna(row['VAT/GST (standard)']):
        return 0
    else:
        # Convert value to string and split
        if '%' in str(row['VAT/GST (standard)']):
            value_string = str(row['VAT/GST (standard)']).split('%')[0]
            return float(value_string) / 100
        else:
            return row['VAT/GST (standard)']

df['vat_cleaned'] = df.apply(clean_vat, axis=1)

In [42]:
df.to_csv('imf_vat_europe.csv', index=False)

Let's combine with TradingEconomics dataset on standard sales tax / vat rates.

In [21]:
# Open .xlsx file
df_te = pd.read_excel('vat_substandard_rates.xlsx', sheet_name='TE')

In [22]:
df_te

,Country,Last,Reference
0,Afghanistan,10.0,2022-12-01
1,Albania,20.0,2023-12-01
2,Algeria,19.0,2023-12-01
3,Andorra,4.5,2022-12-01
4,Angola,14.0,2023-12-01
...,...,...,...
174,Vanuatu,15.0,2023-12-01
175,Venezuela,16.0,2023-12-01
176,Vietnam,10.0,2023-12-01
177,Zambia,16.0,2023-12-01
